In [16]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

In [31]:
%matplotlib inline

In [18]:
%%sql
SELECT count(*) FROM employee CROSS JOIN customer 

1 rows affected.


count
472


In [27]:
%%sql
EXPLAIN ANALYSE
SELECT count(*) FROM employee CROSS JOIN customer;

7 rows affected.


QUERY PLAN
Aggregate (cost=10.77..10.78 rows=1 width=8) (actual time=0.906..0.907 rows=1 loops=1)
-> Nested Loop (cost=0.00..9.59 rows=472 width=0) (actual time=0.041..0.655 rows=472 loops=1)
-> Seq Scan on customer (cost=0.00..2.59 rows=59 width=0) (actual time=0.026..0.061 rows=59 loops=1)
-> Materialize (cost=0.00..1.12 rows=8 width=0) (actual time=0.000..0.005 rows=8 loops=59)
-> Seq Scan on employee (cost=0.00..1.08 rows=8 width=0) (actual time=0.007..0.011 rows=8 loops=1)
Planning time: 0.413 ms
Execution time: 1.257 ms


In [20]:
%%sql
SELECT count(*) FROM employee LEFT JOIN customer ON employee.employeeid = customer.supportrepid;

1 rows affected.


count
64


In [30]:
%%sql
EXPLAIN ANALYSE
SELECT count(*) FROM employee LEFT JOIN customer ON employee.employeeid = customer.supportrepid;

9 rows affected.


QUERY PLAN
Aggregate (cost=4.60..4.61 rows=1 width=8) (actual time=0.783..0.784 rows=1 loops=1)
-> Hash Right Join (cost=1.18..4.45 rows=59 width=0) (actual time=0.082..0.747 rows=64 loops=1)
Hash Cond: (customer.supportrepid = employee.employeeid)
-> Seq Scan on customer (cost=0.00..2.59 rows=59 width=8) (actual time=0.014..0.049 rows=59 loops=1)
-> Hash (cost=1.08..1.08 rows=8 width=8) (actual time=0.048..0.048 rows=8 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 9kB
-> Seq Scan on employee (cost=0.00..1.08 rows=8 width=8) (actual time=0.028..0.035 rows=8 loops=1)
Planning time: 0.413 ms
Execution time: 0.870 ms


In [22]:
%%sql
SELECT count(*) FROM employee FULL JOIN customer ON employee.employeeid = customer.supportrepid;

1 rows affected.


count
64


In [23]:
%%sql
EXPLAIN ANALYSE
SELECT count(*) FROM employee FULL JOIN customer ON employee.employeeid = customer.supportrepid;

9 rows affected.


QUERY PLAN
Aggregate (cost=4.60..4.61 rows=1 width=8) (actual time=0.190..0.190 rows=1 loops=1)
-> Hash Full Join (cost=1.18..4.45 rows=59 width=0) (actual time=0.072..0.163 rows=64 loops=1)
Hash Cond: (customer.supportrepid = employee.employeeid)
-> Seq Scan on customer (cost=0.00..2.59 rows=59 width=8) (actual time=0.009..0.029 rows=59 loops=1)
-> Hash (cost=1.08..1.08 rows=8 width=8) (actual time=0.041..0.041 rows=8 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 9kB
-> Seq Scan on employee (cost=0.00..1.08 rows=8 width=8) (actual time=0.023..0.028 rows=8 loops=1)
Planning time: 0.670 ms
Execution time: 0.299 ms


In [24]:
%%sql
SELECT count(*) FROM employee RIGHT JOIN customer ON employee.employeeid = customer.supportrepid;

1 rows affected.


count
59


In [25]:
%%sql
EXPLAIN ANALYSE
SELECT count(*) FROM employee RIGHT JOIN customer ON employee.employeeid = customer.supportrepid;

4 rows affected.


QUERY PLAN
Aggregate (cost=2.74..2.75 rows=1 width=8) (actual time=0.067..0.067 rows=1 loops=1)
-> Seq Scan on customer (cost=0.00..2.59 rows=59 width=0) (actual time=0.020..0.041 rows=59 loops=1)
Planning time: 0.190 ms
Execution time: 0.135 ms


In [34]:
%%sql
SELECT box '(1,1),(2,2)' - point '(2,0)'

1 rows affected.


?column?
"(2,4),(1,3)"


In [35]:
%%sql
SELECT box '(1,0),(10,10)' # box '(1,1),(2,2)'

1 rows affected.


?column?
"(2,2),(1,1)"


In [36]:
%%sql
SELECT point '0,2' <-> box '(1,3),(-20,0)'

1 rows affected.


?column?
0.0


In [44]:
%%sql
SELECT point(7,7) <@ circle (point(4,2), 5.83)

1 rows affected.


?column?
False


In [45]:
import sqlalchemy
sqlalchemy.__version__

'1.1.13'

In [46]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://appdev@data:5432/appdev', echo=True)

In [47]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [48]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

In [49]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [50]:
Base.metadata.create_all(engine)

2018-04-05 12:45:35,662 INFO sqlalchemy.engine.base.Engine select version()
2018-04-05 12:45:35,666 INFO sqlalchemy.engine.base.Engine {}
2018-04-05 12:45:35,678 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-04-05 12:45:35,681 INFO sqlalchemy.engine.base.Engine {}
2018-04-05 12:45:35,690 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-05 12:45:35,693 INFO sqlalchemy.engine.base.Engine {}
2018-04-05 12:45:35,697 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-05 12:45:35,703 INFO sqlalchemy.engine.base.Engine {}
2018-04-05 12:45:35,710 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-04-05 12:45:35,713 INFO sqlalchemy.engine.base.Engine {}
2018-04-05 12:45:35,718 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [51]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [52]:
ed_user.name

'ed'

In [53]:
ed_user.password

'edspassword'

In [54]:
str(ed_user.id)

'None'

In [55]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [56]:
Session = sessionmaker()

In [57]:
Session.configure(bind=engine)

In [58]:
session = Session()

In [67]:
eds_user = User(name='eds', fullname='Ed Jones', password='edspassword')

In [69]:
session.add(eds_user)

In [76]:
session.rollback()
some_user = session.query(User).filter_by(name='ed').first() 

2018-04-05 12:52:23,890 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-05 12:52:23,900 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = %(name_1)s 
 LIMIT %(param_1)s
2018-04-05 12:52:23,909 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'param_1': 1}


/opt/conda/lib/python3.6/site-packages/sqlalchemy/orm/session.py:507: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


ProgrammingError: (psycopg2.ProgrammingError) column users.id does not exist
LINE 1: SELECT users.id AS users_id, users.name AS users_name, users...
               ^
 [SQL: 'SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password \nFROM users \nWHERE users.name = %(name_1)s \n LIMIT %(param_1)s'] [parameters: {'name_1': 'ed', 'param_1': 1}]

In [64]:
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>